In [1]:
import torch
import sys
from swift.llm import get_model_tokenizer
dirs = ["/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm"]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)
from custom import CustomModelType, CustomTemplateType
model_type = CustomModelType.mixtral_moe_7b_instruct_gptq_int4
model, tokenizer = get_model_tokenizer(
    model_type, 
    load_model=False
)


2024-04-06 14:55:45,936 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-04-06 14:55:45,939 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-04-06 14:55:45,974 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 ad87f4a589251d86333765b92ab59e41 and a total number of 972 components indexed
[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mixtral",
  "num_attention_heads": 32,
  "num_experts_per_tok": 2,
  "num_hidde

In [2]:
import jsonlines
import json
from tqdm.notebook import tqdm


sort = False
prior_knowledge_list = []

K = 5
prior_knowledge_version = "1"
model_name = "mixtral"


with open(f"train_search_llm_tmp.json", "r") as f:
    prior_knowledge_list = json.load(f)


lens1 = []
lens2 = []

# for item in tqdm(prior_knowledge_list):
#     pk=item[f"prior_knowledge_{model_name}"]
#     lens1.append(len(tokenizer(pk)["input_ids"]))

for item in tqdm(prior_knowledge_list):
    if item.get(f"prior_knowledge_{model_name}3") is None:
        lens1.append(1000)
        continue

    pk=item[f"prior_knowledge_{model_name}3"]
    lens1.append(len(tokenizer(pk)["input_ids"]))
    # lens2.append(len(tokenizer.tokenize(item["query"])))


  0%|          | 0/14384 [00:00<?, ?it/s]

In [3]:
# 300以下用llama template

sorted(lens1, reverse=False)[:10]


[205, 207, 221, 224, 230, 234, 239, 255, 276, 279]

修改较长的句子

显卡 24GB

7B模型，长度控制在3600以下 


In [8]:
del_ids = []
for i, item in enumerate(lens1):
    if item == 207:
        del_ids.append(i)

del_ids


[9076]

In [16]:
# print(len(tokenizer(dict_list[4157]["query"])["input_ids"]))
prior_knowledge_list[7299][f"prior_knowledge_{model_name}"]


'Information'

In [10]:
print(prior_knowledge_list[9076][f"prior_knowledge_{model_name}3"])


Information Summary:
The three pieces of information provided all discuss a photo that is being shared on social media, claiming to show a protest in Berlin against coronavirus restrictions. However, the photo is actually of a music festival in Zurich, Switzerland that took place in 2019. The police estimate that 17,000 people attended the protest in Berlin on August 1, while the music festival in Zurich attracted 850,000 people. The mislabeled photo has been shared on social media platforms such as Facebook and Twitter.

Claim Verification:
The claim that the photo shows crowds protesting in Berlin against coronavirus restrictions is false, based on the information provided. The photo actually shows a music festival in Zurich, Switzerland that took place in 2019. This has been confirmed by multiple sources, including fact-checking organizations and the Zurich police. Therefore, the claim that the photo shows a protest in Berlin is not accurate.
